In [13]:
import boto3
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from PIL import Image
import json

In [2]:
sess = boto3.Session(profile_name='ml_predict',region_name='ap-northeast-1')

In [4]:
client = sess.client(service_name='sagemaker-runtime',region_name='ap-northeast-1')

In [10]:
agg = pd.read_csv('agg_revised.csv')
ohe = OneHotEncoder(sparse=False, drop='first')
ohe.fit(agg[['age', 'gender', 'season']])
img_path = 'unique_images/50.png'
img_input = np.array([np.array(Image.open(img_path).resize((128,128)))])
brf_input = ohe.transform([['18-34', 'male', 1]])
brf_input = tf.expand_dims(np.append(brf_input, 30), axis=0)

In [32]:
input_data = {
    'inputs': {
        'input_1': img_input.tolist(), 
        'input_2': brf_input.numpy().tolist()
    }
}

In [33]:
input_json = json.dumps(input_data)

In [44]:
res = client.invoke_endpoint(EndpointName='tf-complex-cnn-2021-07-30-16-20-57-143', 
                       Body=input_json, ContentType='application/json')

In [35]:
res

{'ResponseMetadata': {'RequestId': '7a5ca5cd-a4db-4a9c-ba77-3eede5ea2134',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7a5ca5cd-a4db-4a9c-ba77-3eede5ea2134',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Fri, 30 Jul 2021 18:36:05 GMT',
   'content-type': 'application/json',
   'content-length': '69'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x1f3e4f32d48>}

In [45]:
pred = json.loads(res['Body'].read())

In [46]:
pred

{'outputs': [[2.65237093]]}